In [1]:
import yaml
import logging
from typing import Dict, List
from datetime import datetime
import sys
import http.client
import json
import os
os.chdir('/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src')
from pudu.app.main import *

In [2]:
service = NotificationService()
service.send_notification(
    robot_id="123",
    notification_type="robot_status",
    title="robot is offline",
    content="robot 123 is offline"
)

True

In [2]:
type = 'robot_charging'

In [3]:
def send_change_based_notifications(changes_dict: Dict[str, Dict],
                                 data_type: str, time_range: str = ""):
    """Send notifications only for records that have actual data changes"""
    if not changes_dict:
        print(f"No changes detected for {data_type}, no notifications sent")
    # Group changes by robot_id for notification purposes
    # Send individual notification for each changed record
    for unique_id, change_info in changes_dict.items():
        # Generate notification content for this specific record
        title, content = generate_individual_notification_content(data_type, change_info, time_range)
        print(f'{title}\n{content}')

In [4]:
config = DatabaseConfig("/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/configs/database_config.yaml")
tables = initialize_tables_from_config(config, type)
tables

In [5]:
table = tables[0]

In [6]:
table.query_data_as_df()

/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/rds/utils.py:138: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(f"SELECT * FROM {table_name}", connection)


,id,robot_sn,robot_name,start_time,end_time,duration,initial_power,final_power,power_gain,create_time,status
0,1,811064412050012,USF_CC1,2024-09-19 22:43:53,2024-09-19 22:44:03,0h 00min,100%,100%,+0%,2025-03-25 00:03:20,Done
1,2,811064412050012,USF_CC1,2024-09-19 21:32:42,2024-09-19 21:42:47,0h 10min,100%,100%,+0%,2025-03-25 00:03:20,Done
2,3,811064412050012,USF_CC1,2024-09-19 21:12:11,2024-09-19 21:16:56,0h 04min,100%,100%,+0%,2025-03-25 00:03:20,Done
3,4,811064412050012,USF_CC1,2024-09-19 20:17:24,2024-09-19 20:17:55,0h 00min,100%,100%,+0%,2025-03-25 00:03:20,Done
4,5,811064412050012,USF_CC1,2024-09-19 20:09:51,2024-09-19 20:09:51,0h 00min,100%,100%,+0%,2025-03-25 00:03:20,Done
...,...,...,...,...,...,...,...,...,...,...,...
203,824,8110H4802050005,salt-lake-airport_CC1,2025-07-10 07:53:58,2025-07-10 07:56:10,0h 02min,10%,11%,+1%,2025-07-10 09:00:20,Done
204,825,8110H4802050005,salt-lake-airport_CC1,2025-07-10 07:51:33,2025-07-10 07:56:47,0h 05min,8%,10%,+2%,2025-07-10 09:00:20,Done
205,860,811135422060217,Mobiltech_CC1,2025-07-23 18:54:46,2025-07-23 18:59:55,0h 05min,64%,66%,+2%,2025-07-23 19:11:15,None
206,865,811135422060217,Mobiltech_CC1,2025-07-23 19:24:05,2025-07-23 19:45:47,0h 21min,66%,76%,+10%,2025-07-23 19:36:15,None


In [10]:
data_list = table.query_data_as_df().to_dict(orient='records')

/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/rds/utils.py:138: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(f"SELECT * FROM {table_name}", connection)


In [11]:
# data_list[0]['event_level'] = 'severe'
# data_list[2]['event_level'] = 'severe'
# data_list[0]['progress'] +=1
data_list[1]['initial_power'] = '123'
data_list[2]['initial_power'] = '321'
# new_entries = [data_list[0].copy(), data_list[1].copy()]
# new_entries[0]['robot_sn'] = '123'
# new_entries[1]['robot_sn'] = '321'
# data_list = new_entries + data_list

changes = detect_data_changes(table, data_list, table.primary_keys)
changes

{'811064412050012_2024-09-19 21:32:42_2024-09-19 21:42:47': {'robot_id': '811064412050012',
  'primary_key_values': {'robot_sn': '811064412050012',
   'start_time': Timestamp('2024-09-19 21:32:42'),
   'end_time': Timestamp('2024-09-19 21:42:47')},
  'change_type': 'update',
  'changed_fields': ['initial_power'],
  'old_values': {'id': 2,
   'robot_sn': '811064412050012',
   'robot_name': 'USF_CC1',
   'start_time': datetime.datetime(2024, 9, 19, 21, 32, 42),
   'end_time': datetime.datetime(2024, 9, 19, 21, 42, 47),
   'duration': '0h 10min',
   'initial_power': '100%',
   'final_power': '100%',
   'power_gain': '+0%',
   'create_time': datetime.datetime(2025, 3, 25, 0, 3, 20),
   'status': 'Done'},
  'new_values': {'id': 2,
   'robot_sn': '811064412050012',
   'robot_name': 'USF_CC1',
   'start_time': Timestamp('2024-09-19 21:32:42'),
   'end_time': Timestamp('2024-09-19 21:42:47'),
   'duration': '0h 10min',
   'initial_power': '123',
   'final_power': '100%',
   'power_gain': '+0%'

In [12]:
send_change_based_notifications(changes, type, '2025-07-23 00:00:00')

Charging Session Updated
Robot 811064412050012 charging session (started 2024-09-19 21:32:42) updated: initial_power
Charging Session Updated
Robot 811064412050012 charging session (started 2024-09-19 21:12:11) updated: initial_power
